## Перша частина - логи з CLI


### 2. Перевірити правильність конфігурації за допомогою 

```bash
root@21492bb80ba1:/# nodetool status
Datacenter: datacenter1

Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack
UN  172.24.0.4  103.87 KiB  16      59.3%             3810d6d2-4e44-4cdf-9759-6ae3dc8294f2  rack1
UN  172.24.0.2  138.62 KiB  16      64.7%             6899404e-db8f-4e19-9e2a-ed329185a070  rack1
UN  172.24.0.3  75.28 KiB   16      76.0%             dd40b402-91f5-4c3b-9065-2bf8cd7cfcd8  rack1
```

### 3. Викоритовуючи *cqlsh*, створити три *Keyspace* з replication factor 1, 2, 3

```bash
cqlsh> CREATE KEYSPACE IF NOT EXISTS keyspace_1 WITH REPLICATION={'class' : 'SimpleStrategy', 'replication_factor': 1};
cqlsh> CREATE KEYSPACE IF NOT EXISTS keyspace_2 WITH REPLICATION={'class' : 'SimpleStrategy', 'replication_factor': 2};
cqlsh> CREATE KEYSPACE IF NOT EXISTS keyspace_3 WITH REPLICATION={'class' : 'SimpleStrategy', 'replication_factor': 3};
```

### 4. В кожному з кейспейсів створити таблиці


```bash
cqlsh> CREATE TABLE IF NOT EXISTS keyspace_1.items ( category_name text, price         float, item_id       int, PRIMARY KEY (category_name, price, item_id));
cqlsh> CREATE TABLE IF NOT EXISTS keyspace_2.items ( category_name text, price         float, item_id       int, PRIMARY KEY (category_name, price, item_id));
cqlsh> CREATE TABLE IF NOT EXISTS keyspace_3.items ( category_name text, price         float, item_id       int, PRIMARY KEY (category_name, price, item_id));
```

### 5. Спробуйте писати і читати на / та з різних нод.

In [269]:
# !pip install cassandra-driver

from cassandra.cluster import Cluster

cluster_1 = Cluster(['127.0.0.1'],port=9042)
cluster_2 = Cluster(['127.0.0.1'],port=9043)
cluster_3 = Cluster(['127.0.0.1'],port=9044)

In [73]:
session_1 = cluster_1.connect('keyspace_3')

session_2 = cluster_2.connect('keyspace_3')

session_3 = cluster_3.connect('keyspace_3')


queries_for_insert = [
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (4, 850.00, 'Phones');''',
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (5, 1150.00, 'Phones');''',
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (6, 1850.00, 'Phones');''']

# Спроба підключитися на першу ноду
print(session_1.execute(queries_for_insert[0]))
    
# Спроба підключитися на другу ноду
print(session_2.execute(queries_for_insert[1]))

# Спроба підключитися на третю ноду
print(session_3.execute(queries_for_insert[2]))

In [7]:
query_to_find_special_category = '''
SELECT *
FROM keyspace_1.items
WHERE category_name = 'Phones'
'''

for row in session_1.execute(query_to_find_special_category):
    print(row)

Row(category_name='Phones', price=850.0, item_id=1)
Row(category_name='Phones', price=850.0, item_id=4)
Row(category_name='Phones', price=1150.0, item_id=2)
Row(category_name='Phones', price=1150.0, item_id=5)
Row(category_name='Phones', price=1850.0, item_id=3)
Row(category_name='Phones', price=1850.0, item_id=6)


### 6. Вставте дані в створені таблиці і подивіться на їх розподіл по вузлах кластера (для кожного з кейспесов - nodetool status)

In [8]:
queries_for_insert = [
'''INSERT INTO items (item_id, price, category_name)
VALUES (4, 850.00, 'Phones');''',
'''INSERT INTO items (item_id, price, category_name)
VALUES (5, 1150.00, 'Phones');''',
'''INSERT INTO items (item_id, price, category_name)
VALUES (6, 1850.00, 'Phones');''']

session_1.execute('USE keyspace_2')

for insert_query in queries_for_insert:
    print(session_1.execute(insert_query))
    
    
session_1.execute('USE keyspace_3')

for insert_query in queries_for_insert:
    print(session_1.execute(insert_query))

### Росподіл по вузлах

```bash
root@21492bb80ba1:/# nodetool status keyspace_1
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack
UN  172.24.0.4  152.16 KiB  16      31.6%             3810d6d2-4e44-4cdf-9759-6ae3dc8294f2  rack1
UN  172.24.0.2  186.79 KiB  16      32.7%             6899404e-db8f-4e19-9e2a-ed329185a070  rack1
UN  172.24.0.3  75.88 KiB   16      35.7%             dd40b402-91f5-4c3b-9065-2bf8cd7cfcd8  rack1

root@21492bb80ba1:/# nodetool status keyspace_2
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack
UN  172.24.0.4  152.16 KiB  16      59.3%             3810d6d2-4e44-4cdf-9759-6ae3dc8294f2  rack1
UN  172.24.0.2  186.79 KiB  16      64.7%             6899404e-db8f-4e19-9e2a-ed329185a070  rack1
UN  172.24.0.3  75.88 KiB   16      76.0%             dd40b402-91f5-4c3b-9065-2bf8cd7cfcd8  rack1

root@21492bb80ba1:/# nodetool status keyspace_3
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack
UN  172.24.0.4  152.16 KiB  16      100.0%            3810d6d2-4e44-4cdf-9759-6ae3dc8294f2  rack1
UN  172.24.0.2  186.79 KiB  16      100.0%            6899404e-db8f-4e19-9e2a-ed329185a070  rack1
UN  172.24.0.3  75.88 KiB   16      100.0%            dd40b402-91f5-4c3b-9065-2bf8cd7cfcd8  rack1

```

### 7. Для якогось запису з кожного з кейспейсу виведіть ноди на яких зберігаються дані


```bash
root@21492bb80ba1:/# nodetool getendpoints keyspace_1 items 2;
172.24.0.2
root@21492bb80ba1:/# nodetool getendpoints keyspace_2 items 2;
172.24.0.2
172.24.0.4
root@21492bb80ba1:/# nodetool getendpoints keyspace_3 items 2;
172.24.0.2
172.24.0.4
172.24.0.3
root@21492bb80ba1:/#
```

### 8. Відключиіть одну з нод. Для кожного з кейспейсів визначить з якими рівнями consistency можемо читати та писати, і які з них забезпечують strong consistency 

##### Перевіремо CONSISTENCY level встановленний на кластері
```bash
cqlsh> CONSISTENCY
Current consistency level is ONE.
cqlsh>
```

#### В нас зараз такі умови
- Current consistency level is ONE.
- Одна з трех нод відключена
- keyspace_1 має Replication Factor - 1
- keyspace_2 має Replication Factor - 2
- keyspace_3 має Replication Factor - 3

Тому strong consistency зараз не маємо ніде

Strong consistency ми можемо забеспечити лише для keyspace_3 з Replication Factor - 3, якщо зробимо CONSISTENCY TWO OR QUORUM

### 9. Зробить так щоб три ноди працювали, але не бачили одна одну по мережі (відключити зв'язок між ними)

### Виключив за допомогою 
- nodetool disablegossip

### 10. Для кейспейсу з replication factor 3 задайте рівень consistency рівним 1. Виконайте запис одного й того самого значення, з однаковим primary key, але різними іншими значенням на кожну з нод (тобто створіть конфлікт)


In [115]:
session_1 = cluster_1.connect('keyspace_3')

queries_for_insert = [
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (4, 1850.00, 'TV');''']

# Записую на першу ноду
print(session_1.execute(queries_for_insert[0]))

In [116]:
session_2 = cluster_2.connect('keyspace_3')

queries_for_insert = [
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (4, 21850.00, 'TV');''']

# Записую на першу ноду
print(session_2.execute(queries_for_insert[0]))

In [117]:
session_3 = cluster_3.connect('keyspace_3')

queries_for_insert = [
'''INSERT INTO keyspace_3.items (item_id, price, category_name)
VALUES (4, 31850.00, 'TV');''']

# Записую на першу ноду
print(session_3.execute(queries_for_insert[0]))

In [119]:
query_to_find_special_category = '''
SELECT *
FROM keyspace_3.items
WHERE category_name = 'TV'
AND item_id = 4
'''

for row in session_1.execute(query_to_find_special_category):
    print(row)    
print('\n')

for row in session_2.execute(query_to_find_special_category):
    print(row)
print('\n')
for row in session_3.execute(query_to_find_special_category):
    print(row)

Row(category_name='TV', item_id=4, price=1850.0)


Row(category_name='TV', item_id=4, price=21850.0)


Row(category_name='TV', item_id=4, price=31850.0)


### 11. Об’єднайте ноди в кластер і визначте яке значення було прийнято кластером та за яким принципом

### Для цього включу знов nodetool enablegossip

In [125]:
query_to_find_special_category = '''
SELECT *
FROM keyspace_3.items
WHERE category_name = 'TV'
AND item_id = 4
'''

for row in session_1.execute(query_to_find_special_category):
    print(row)    
print('\n')

for row in session_2.execute(query_to_find_special_category):
    print(row)
print('\n')
for row in session_3.execute(query_to_find_special_category):
    print(row)

Row(category_name='TV', item_id=4, price=31850.0)


Row(category_name='TV', item_id=4, price=31850.0)


Row(category_name='TV', item_id=4, price=31850.0)


### Результат
- Спочатку об'єднались 1 та 2 ноди, і перша прийняла значення 21850
- Потім до кластеру під'єдналась 3 нода, і всюди значення стало 31850

### Це відбувається через стратегію Cassandra - Last Write Wins, LWW 

### 12. Перевірте поведінку lightweight transactions для попередніх пунктів у розділеному та не розділеному на три частини кластері https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertLWT.html 


In [286]:
session_1 = cluster_1.connect('keyspace_3')

queries_for_insert = [
'''UPDATE keyspace_3.items
  SET price = 1100
  WHERE category_name = 'TV'
    AND item_id = 4
  IF price > 1000;''']

# Записую на першу ноду
print(session_1.execute(queries_for_insert[0]))

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level SERIAL" info={\'consistency\': \'SERIAL\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

### Неможливо застосувати, так як required replicas - 2 та required consistency level SERIAL

### Якщо зв'язати кластер - то результат буде наступний

In [294]:
session_1 = cluster_1.connect('keyspace_3')

queries_for_insert = [
'''UPDATE keyspace_3.items
  SET price = 1100
  WHERE category_name = 'TV'
    AND item_id = 4
  IF price > 1000;''']

# Записую на першу ноду
print(session_1.execute(queries_for_insert[0]))


for row in session_1.execute(query_to_find_special_category):
    print(row)    
print('\n')
for row in session_2.execute(query_to_find_special_category):
    print(row)    
print('\n')
for row in session_3.execute(query_to_find_special_category):
    print(row)    
print('\n')

Row(category_name='TV', item_id=4, price=1100.0)


Row(category_name='TV', item_id=4, price=1100.0)


Row(category_name='TV', item_id=4, price=1100.0)


